Test the methods for the second day phase 1

In [30]:
import os
import requests
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
#from distance_airports import distance_geo
from plotnine import ggplot, aes, geom_histogram, theme_minimal, scale_fill_manual
import geopandas as gpd
import matplotlib.pyplot as plt
from difflib import get_close_matches
import random

class Airplane():
    """
    A class to download airplane data and perform analysis on it.
    """

    def __init__(self):
        self.airlines_df = pd.DataFrame()
        self.airplanes_df = pd.DataFrame()
        self.airports_df = pd.DataFrame()
        self.routes_df = pd.DataFrame()
        self.merge_df = pd.DataFrame()

    def download_data(self):
        """
        Checks for a 'downloads' folder and creates it if it doesn't exist.
        Downloads flight data from a GitHub repository as a zip file, saves it,
        extracts the contents, and creates Pandas DataFrames from the CSV files:
        - airlines.csv
        - airplanes.csv
        - airports.csv
        - routes.csv
        """

        downloads_dir = "downloads"
        if not os.path.exists(downloads_dir):
            os.makedirs(downloads_dir)

        zip_file_path = os.path.join(downloads_dir, "flight_data.zip")

        """
        Check if the zip file was already downloaded
        """
        if not os.path.exists(zip_file_path):
            response = requests.get(
                "https://gitlab.com/adpro1/adpro2024/-/"
                "raw/main/Files/flight_data.zip?inline=false",
                stream=True,
            )
            """
            Ensure the request is successful
            """
            if response.status_code == 200:
                with open(zip_file_path, "wb") as f:
                    f.write(response.content)

        """
        Extract files if the zip file is downloaded
        """
        if os.path.exists(zip_file_path):
            extraction_dir = os.path.join(os.getcwd(), downloads_dir)
            with ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(downloads_dir)

                """
                Loading data into DataFrames
                """
                self.airlines_df = pd.read_csv(
                    os.path.join(downloads_dir, "airlines.csv")
                )
                self.airplanes_df = pd.read_csv(
                    os.path.join(downloads_dir, "airplanes.csv")
                )
                self.airports_df = pd.read_csv(
                    os.path.join(downloads_dir, "airports.csv")
                )
                self.routes_df = pd.read_csv(os.path.join(downloads_dir, "routes.csv"))
        
        """
        Print the first 5 lines of each dataset
        """
        #print("Airlines DataFrame:\n", self.airlines_df.head())
        #print("\nAirplanes DataFrame:\n", self.airplanes_df.head())
        #print("\nAirports DataFrame:\n", self.airports_df.head())
        #print("\nRoutes DataFrame:\n", self.routes_df.head())

        #print("\nDownloaded data executed!")
        
    def merge_datasets(self) -> pd.DataFrame:
    
        self.airlines_df = self.airlines_df.drop(self.airlines_df.columns[0], axis=1).reset_index(drop=True)
        self.airports_df= self.airports_df.drop(self.airports_df.columns[0], axis=1).reset_index(drop=True)
        self.airports_df = self.airports_df.drop(['Type', 'Source'], axis=1)
        self.routes_df= self.routes_df.drop(self.routes_df.columns[0], axis=1).reset_index(drop=True)
        self.airplanes_df= self.airplanes_df.drop(self.airplanes_df.columns[0], axis=1).reset_index(drop=True)

        merge_df_1 = pd.merge(self.airports_df,self.routes_df, left_on="IATA", right_on="Source airport", how="left")
        merge_df_1 = merge_df_1.rename(columns={"Country": "Source country","Latitude": "latitude_source","Longitude": "longitude_source"})
        
        merge_df = pd.merge(merge_df_1,self.airports_df[["IATA", "Country", "Latitude", "Longitude"]],left_on="Destination airport",right_on="IATA", how='left')
        merge_df = merge_df.rename(columns={'IATA_x':"IATA", "Country": "Destination country","Latitude": "latitude_destination","Longitude": "longitude_destination"})
        merge_df = merge_df.dropna(subset=["Source country", "Destination country"])
        #Drop the columns that I dont need
        merge_df.drop(columns=['IATA_y'], inplace=True)

        # Assign the resulting merge_df to the instance variable
        self.merge_df = merge_df
        
        #print("Merge DataFrame:\n", self.merge_df.head())

        return self.merge_df

    def plot_airports_in_country(self, country):
        """
        Plot airports within the chosen country and use the ICAOs as legends.

        Parameters:
        - country (str): The name of the country where airports are to be plotted.
        """
        # Filter airports by the specified country
        country_airports = self.merge_df[self.merge_df["Source country"] == country]

        # Exit if no airports are found in the country
        if country_airports.empty:
            print(f"No airports found in {country}.")
            return

        # Load a map of the world
        world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

        # Filter the map to only include the specified country
        country_map = world[world["name"] == country]

        # Exit if the country is not found in the map dataset
        if country_map.empty:
            print(f"Country '{country}' not found.")
            return

        # Convert the filtered airports data into a GeoDataFrame
        gdf_airports = gpd.GeoDataFrame(
            country_airports,
            geometry=gpd.points_from_xy(
                country_airports.Longitude, country_airports.Latitude
            ),
        )

        # Plotting
        _, axis = plt.subplots(figsize=(10, 10))
        country_map.plot(ax=axis, color="lightgrey")
        gdf_airports.plot(ax=axis, marker="o", color="red", markersize=5)

        # Annotate each airport with its ICAO code
        for _, row in gdf_airports.iterrows():
            axis.text(row.geometry.x, row.geometry.y, row["ICAO"], fontsize=8)

        plt.title(f"Airports in {country}")
        plt.show()

    def distance_analysis(self):
        self.merge_df["distance"] = self.merge_df.apply(
            lambda row: distance_geo(
                row["latitude_source"],
                row["longitude_source"],
                row["latitude_destination"],
                row["longitude_destination"],
            ),
            axis=1,
        )

        self.merge_df["distance"].head()

        distance_plot = (
            ggplot(self.merge_df, aes(x="distance"))
            + geom_histogram(bins=30, fill="#5496BF", color="black")
            + theme_minimal()
            + scale_fill_manual(
                values=["#011526", "#C9DFF2", "#5496BF", "#75B2BF", "#025159"]
            )
        )
        return distance_plot

    def plot_flights_by_code_airports(self, code_airport, n=5, internal=False):

        country_of_source = self.merge_df.loc[
            self.merge_df["Source airport"] == code_airport, "Source country"
        ].iloc[0]

        if internal:
            # Plot only internal flights
            internal_routes = self.merge_df[
                (self.merge_df["Source airport"] == code_airport)
                & (self.merge_df["Destination airport"] == code_airport)
            ]
            # Limit to top n destination airports
            internal_routes["Destination airport"].value_counts().head(n).plot(
                kind="bar", title=f"Top {n} Internal Flights from {code_airport}"
            )
        else:
            # Plot all flights
            all_routes = self.merge_df[self.merge_df["Source airport"] == code_airport]
            # Limit to top n destination airports
            all_routes["Destination airport"].value_counts().head(n).plot(
                kind="bar", title=f"Top {n} Flights from {code_airport}"
            )
        plt.xlabel("Destination Airport")
        plt.ylabel("Number of Flights")
        plt.show()

    def plot_most_used_airplane_models(self, n: int = 5, countries=None):
        """
        Plot the most used airplane models based on the number of routes.

        Parameters:
            n (int): Number of airplane models to plot, that by default is 5.
            countries: Specific country or list of countries to consider, by default is None.
        """
        if countries is None:
            # Plot most used planes by route using the entire dataset
            top_airplanes = self.merge_df["Equipment"].value_counts().nlargest(n)
        else:
            # Plot most used planes by route for a specific country or list of countries
            if isinstance(countries, str):
                countries = [countries]

            filtered_routes = self.merge_df[
                self.merge_df["Source country"].isin(countries)
            ]
            top_airplanes = filtered_routes["Equipment"].value_counts().nlargest(n)

        top_airplanes.plot(
            kind="bar", title=f"Top {n} Airplane Models by Number of Routes"
        )
        plt.xlabel("Airplane Model")
        plt.ylabel("Number of Routes")
        plt.show()

    def plot_flights_by_country(self, country_name, internal=False):
        """
        Plot flights leaving the country with a destination in the same country or all flights.

        Parameters:
            country_name (str): The name of the country.
            internal (bool): If True, plot only internal flights. If False, plot all flights. Default is False.
        """
        if internal is True:
            # Plot only internal flights
            internal_routes = self.merge_df[
                (self.merge_df["Source country"] == country_name)
                & (self.merge_df["Destination country"] == country_name)
            ]
            internal_routes["Destination country"].value_counts().plot(
                kind="bar", title=f"Internal Flights from {country_name}"
            )
        else:
            # Plot all flights
            all_routes = self.merge_df[
                (self.merge_df["Source country"] == country_name)
            ]
            all_routes["Destination country"].value_counts().plot(
                kind="bar", title=f"All Flights from {country_name}"
            )

        plt.xlabel("Airplane Model")
        plt.ylabel("Number of Flights")
        plt.show()
        
    # DAY2, PHASE 1, METHOD 1   
    def aircrafts(self):
        column_names = self.airplanes_df.columns
        print("Column names in airplanes_df:", column_names)
        # Assuming 'Name' is the column containing aircraft models in self.airplane_df
        aircraft_models = self.airplanes_df['Name'].tolist()
        print("List of Aircraft Models:")
        for model in aircraft_models:
            print(model)
       
    # DAY2, PHASE 1, METHOD 2
    def aircraft_info(self, aircraft_name):
        aircraft_models = self.airplanes_df['Name'].tolist()

        # Check if the provided aircraft_name is in the list
        if aircraft_name in aircraft_models:
            # Retrieve information for the specified aircraft_name
            info = self.airplanes_df[self.airplanes_df['Name'] == aircraft_name][['Name', 'IATA code', 'ICAO code']]
            print(f"Aircraft Information for {aircraft_name}:")
            print(info.to_string(index=False))
        else:
            # Attempt to find close matches
            close_matches = get_close_matches(aircraft_name, aircraft_models, n=5, cutoff=0.3)
            if close_matches:
                print(f"Aircraft '{aircraft_name}' not found. Did you mean:")
                for match in close_matches:
                    print(f"- {match}")
            else:
                # If no close matches found, suggest a generic list of recommended aircraft names
                print(f"No close matches found for '{aircraft_name}'.")
                
                # Shuffle the list of aircraft names to ensure a different subset is selected each time
                all_aircraft_names = self.airplanes_df['Name'].tolist()
                random.shuffle(all_aircraft_names)  # Shuffle the list in place

                # Select the top 5 (or however many you prefer) from the shuffled list
                recommended_list = all_aircraft_names[:5]
                print("Here are some recommended aircraft names to choose from:")
                for recommend in recommended_list:
                    print(f"- {recommend}")
            
     # DAY2, PHASE 1, METHOD 4       
    def airport_info(self, airport_name):
        airport_models = self.merge_df['Name'].tolist()

        # Check if the provided airport_name is in the list
        if airport_name in airport_models:
            # Retrieve information for the specified airport_name
            unique_info = self.merge_df[self.merge_df['Name'] == airport_name][['Airport ID', 'Name', 'Source airport', 'City', 'latitude_source', 'longitude_source']].drop_duplicates(subset=['Name'])
            print(f"Airport Information for {airport_name}:")
            print(unique_info.to_string(index=False))
        else:
            # Attempt to find close matches
            close_matches = get_close_matches(airport_name, airport_models, n=5, cutoff=0.3)
            if close_matches:
                print(f"Aircraft '{airport_name}' not found. Did you mean:")
                for match in close_matches:
                    print(f"- {match}")
            else:
                # If no close matches found, suggest a generic list of recommended aircraft names
                print(f"No close matches found for '{airport_name}'.")
                
                # Shuffle the list of aircraft names to ensure a different subset is selected each time
                all_airport_names = self.merge_df['Name'].tolist()
                random.shuffle(all_airport_names)  # Shuffle the list in place

                # Select the top 5 (or however many you prefer) from the shuffled list
                recommended_list = all_airport_names[:5]
                print("Here are some recommended aircraft names to choose from:")
                for recommend in recommended_list:
                    print(f"- {recommend}")


In [31]:
# Assuming you have already created an instance of the Airplane class
airplane_instance = Airplane()
airplane_instance.download_data()
airplane_instance.merge_datasets()

,Airport ID,Name,City,Source country,IATA,ICAO,latitude_source,longitude_source,Altitude,Timezone,...,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Destination country,latitude_destination,longitude_destination
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,...,GKA,1,HGU,3,NaN,0.0,DH8 DHT,Papua New Guinea,-5.826790,144.296005
1,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,...,GKA,1,LAE,4,NaN,0.0,DH8,Papua New Guinea,-6.569803,146.725977
2,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,...,GKA,1,MAG,2,NaN,0.0,DH8,Papua New Guinea,-5.207080,145.789001
3,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,...,GKA,1,POM,5,NaN,0.0,DH8,Papua New Guinea,-9.443380,147.220001
4,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,...,GKA,1,POM,5,NaN,0.0,DH4 DH8 DH3,Papua New Guinea,-9.443380,147.220001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71691,14102,Laguindingan Airport,Cagayan de Oro City,Philippines,CGY,RPMY,8.612203,124.456496,190,\N,...,CGY,2400,MNL,2397,NaN,0.0,320,Philippines,14.508600,121.019997
71698,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,...,CPO,4312,CJC,2645,NaN,0.0,319,Chile,-22.498199,-68.903603
71699,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,...,CPO,4312,LSC,2662,NaN,0.0,319,Chile,-29.916201,-71.199501
71700,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,...,CPO,4312,SCL,2650,NaN,0.0,320 319,Chile,-33.393002,-70.785797


In [32]:
# Test with a valid airport name
try:
    airplane_instance.airport_info("Goroka Airport")
except ValueError as e:
    print(e)

# Test with an invalid airport name
try:
    airplane_instance.airport_info("Larg")
except ValueError as e:
    print(e)

# Test with an blank airport name
try:
    airplane_instance.airport_info("")
except ValueError as e:
    print(e)

Airport Information for Goroka Airport:
 Airport ID           Name Source airport   City  latitude_source  longitude_source
          1 Goroka Airport            GKA Goroka         -6.08169        145.391998
Aircraft 'Larg' not found. Did you mean:
- Lar Airport
- Lar Airport
- Lar Airport
- Lar Airport
- Lar Airport
No close matches found for ''.
Here are some recommended aircraft names to choose from:
- Marseille Provence Airport
- Norman Y. Mineta San Jose International Airport
- Syktyvkar Airport
- Berlin-Schönefeld Airport
- Al Maktoum International Airport


In [12]:
#Test the method Aircraft
# Call the aircrafts method to print the list of aircraft models
airplane_instance.aircrafts()

Column names in airplanes_df: Index(['Name', 'IATA code', 'ICAO code'], dtype='object')
List of Aircraft Models:
Aerospatiale (Nord) 262
Aerospatiale (Sud Aviation) Se.210 Caravelle
Aerospatiale SN.601 Corvette
Aerospatiale/Alenia ATR 42-300
Aerospatiale/Alenia ATR 42-500
Aerospatiale/Alenia ATR 42-600
Aerospatiale/Alenia ATR 72
Airbus A300
Airbus A300-600
Airbus A300-600ST Super Transporter / Beluga
Airbus A310
Airbus A318
Airbus A319
Airbus A319neo
Airbus A320
Airbus A320neo
Airbus A321
Airbus A321neo
Airbus A330
Airbus A330-200
Airbus A330-300
Airbus A330-700 Beluga XL
Airbus A330-800neo
Airbus A330-900neo
Airbus A340
Airbus A340-200
Airbus A340-300
Airbus A340-500
Airbus A340-600
Airbus A350
Airbus A350-1000
Airbus A350-900
Airbus A380
Airbus A380-800
Antonov AN-12
Antonov AN-124 Ruslan
Antonov AN-140
Antonov An-148
Antonov An-158
Antonov An-225 Mriya
Antonov AN-24
Antonov AN-26
Antonov AN-28
Antonov AN-30
Antonov AN-32
Antonov AN-72
Avro RJ100
Avro RJ70
Avro RJ85
BAe 146
BAe 146-1

In [17]:
# Test with a valid aircraft name
try:
    airplane_instance.aircraft_info("Boeing 747")
except ValueError as e:
    print(e)

# Test with an invalid aircraft name
try:
    airplane_instance.aircraft_info("Air")
except ValueError as e:
    print(e)

# Test with an invalid aircraft name
try:
    airplane_instance.aircraft_info("")
except ValueError as e:
    print(e)

Aircraft Information for Boeing 747:
      Name IATA code ICAO code
Boeing 747       747        \N
Aircraft 'Air' not found. Did you mean:
- Airbus A380
- Airbus A350
- Airbus A340
- Airbus A330
- Airbus A321
No close matches found for ''.
Here are some recommended aircraft names to choose from:
- Embraer Phenom 100
- Airbus A318
- Boeing 757-300
- Sukhoi Superjet 100-95
- Boeing 777
